In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import radarkit
import matplotlib.pyplot as plt
import time

In [ ]:
sweep = radarkit.read('/Users/boonleng/Downloads/PX-20170220-050706-E2.4-Z.nc')

In [ ]:
z = sweep['moments'][0]['data']
r = sweep['moments'][5]['data']

In [ ]:
z.shape

In [ ]:
# Undo range correction and the ZCal in DSP
rr = 1.0e-3 * (np.arange(sweep['gateCount']) + 0.5) * sweep['gateSizeMeters']
snr = z - 20.0 * np.log10(rr) + 23

# Transition gate at 10.5 km at the given spacing
g = int((69.0e-6 * 3.0e8 * 0.5) / sweep['gateSizeMeters'])
print(g)
z_off = 10.0 * np.log10(1.5 / 67)
snr[:, :g] = snr[:, :g] + z_off

In [ ]:
hl = plt.plot(snr.T)

In [ ]:
plt.imshow(snr)
plt.clim(-10, 60)
plt.colorbar()

In [ ]:
# Map NAN to some finite numbers for SNR and PhiDP
snr = np.nan_to_num(snr, -100.0)
r = np.nan_to_num(r, 0.0)

### Fast Code

```python
s = time.time()
for _ in range(10):
    
    mask = np.logical_and(snr>0, r>0.85)

e = time.time()
print(e - s)
```

### Naive Code

```python
s = time.time()
for _ in range(10):
    
    mask = np.zeros(snr.shape, dtype=bool)
    for j in range(360):
        for i in range(snr.shape[1]):
            if (snr[j, i] > 0 and r[j, i] > 0.85):
                mask[j, i] = True

e = time.time()
print(e - s)
```

In [ ]:
mask = np.logical_and(snr>0, r>0.85)

In [ ]:
plt.imshow(mask)

In [ ]:
import random
x = np.arange(10)
x = [random.random() for ii in x]
x

In [ ]:
np.mean(np.diff(x))